In [1]:
import numpy as np
import importlib
import nbimporter

## ファイルから物性値を取得し、流量計算に適した形に換算するモジュール

### 1. 物性値ファイルをインポートするモジュール

#### 1.1 物性名とモジュール名の結びつけ  
プログラム内名称　：　ファイル名  
BenteimerSandstone　：　bentheimer_sandstone  
Concrete　：　Concrete_Goran

In [2]:
def getModuleName( name ):
    try:
        return {
            'BentheimerSandstone': 'bentheimer_sandstone',
            'Concrete': 'concrete_goran',
        }[name]
    except:
        raise ValueError('物性リストに登録されていない物性名が指定されました。')    

#### 1.2 インポートすべきモジュール名をもとに物性値計算を定義した外部ファイルをインポートする

In [3]:
def importModule( moduleName ):
    try:
        return importlib.import_module( moduleName )
    except:
        raise ImportError('物性リストに登録されている外部モジュールが見つかりません。')

In [4]:
# 例
importModule( 'bentheimer_sandstone' )

Importing Jupyter notebook from bentheimer_sandstone.ipynb
Importing Jupyter notebook from liquid_water.ipynb
Importing Jupyter notebook from gas_water.ipynb
Importing Jupyter notebook from moisture_conversion.ipynb
Importing Jupyter notebook from vapour_pressure.ipynb
Importing Jupyter notebook from property_conversion.ipynb
Importing Jupyter notebook from van_genuchten.ipynb


<module 'bentheimer_sandstone' (<nbimporter.NotebookLoader object at 0x000001C815BEA7F0>)>

### 2. 取得した物性値を計算用に換算するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。  

#### 実験から熱水分物性値として考えられるもの  
【参考】建築材料の熱・空気・湿気物性値，日本建築学会，丸善株式会社，2001など

### 2.1 材料実質部の空隙率・密度・比熱などの物性値を取得するクラス

In [11]:
class Kernel_MaterialProperty():
    # 材料実質部の空隙・密度・比熱などの情報を提供するクラス
    
    def __init__( self, name ):
        
        # 物性名称からインポートすべきモジュール名称を検索する。
        moduleName = getModuleName( name )
        
        # インポートすべきモジュールをインポートする。モジュールが存在しなければエラー。
        module = importModule( moduleName )
        
        if ( 'getCRow' in dir( module ) ):
            self.getCRow = module.getCRow
        else:
            # どちらも存在しない場合は関数を追加しない。
            pass
        
        if ( 'psi' in dir( module ) ):
            self.getPsi = module.psi
        else:
            pass
        
        if ( 'row' in dir( module ) ):
            self.getRow = module.row
        else:
            pass
        
        if ( 'C' in dir( module ) ):
            self.getC = module.C
        else:
            pass

In [12]:
# 使用例
a = Kernel_MaterialProperty('Concrete')

Importing Jupyter notebook from concrete_goran.ipynb


In [13]:
a.getPsi()

0.392

### 2.2 材料の水分特性（吸着力）などの情報を取得するクラス

In [14]:
class Kernel_Material_WaterAdsorption():
    # 材料実質部の空隙・密度・比熱などの情報を提供するクラス
    
    def __init__( self, name ):
        
        # 物性名称からインポートすべきモジュール名称を検索する。
        moduleName = getModuleName( name )
        
        # インポートすべきモジュールをインポートする。モジュールが存在しなければエラー。
        module = importModule( moduleName )
        
        if( 'getPhi' in dir( module ) ):
            self.getPhi = module.getPhi
        else:
            pass
        
        if( 'getDPhi' in dir( module ) ):
            self.getDPhi = module.getDPhi
        else:
            pass

        if( 'getMiuByPhi' in dir( module ) ):
            self.getMiuByPhi = module.getMiuByPhi
        else:
            pass

### 2.3 材料特性・含水状態からの熱・水分の移動係数を取得・換算するクラス

#### 熱物性値  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\rho_0$：密度[kg/m3]  
$c_0$：比熱[J/kg K]  
$c_0\rho_0$：容積熱容量[J/m3 K]  

$\lambda$：熱伝導率[W/mK]  
定義：$\dot q = -\lambda・grad\space T$  

$R$：熱抵抗[Km2/W]  

#### 水分物性値  
$\dot m_v$：単位時間当たりの水蒸気流量の密度[kg/m2 s]  
$\dot m_m$：単位時間当たりの水分流量の密度[kg/m2 s]  
$\phi$：含水率[m3/m3]  
他標記として$w$[kg/m3]、$u$[kg/kg]など  
$S$：飽和度[-]  

#### 気相水分伝導率  
$\delta_p$：透湿率[kg/m Pa s]　文中記号DP  
定義：$m_v = -\delta_p・grad\space p_v$  

$Z_p$：湿気抵抗[m2Pa s/kg]  
$\delta_v$：水蒸気の拡散係数[m2/s]  
定義：$\dot m_v = -\delta_v・grad\space \rho_v$

$k_T$：温度勾配水分伝導率[kg/m K s]  
定義：$\dot m_m = -k_T・grad\space T$  

$D_T$：温度勾配水分拡散係数[m2/K s]  
定義：$\dot m_m = -\rho_0 D_T・grad\space T$  

$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu g}・grad\space \mu$

$\lambda^{'}_{Tg}$：温度勾配に対する気相水分伝導率[kg/ms K]  
定義：$\dot m_v = -\lambda^{'}_{Tg}・grad\space \mu$

#### 液相水分伝導率  
$k_m$：水分伝導率[kg/m Pa s]  
定義：$\dot m_m = -k_m・grad\space s$

$D_w$：水分拡散係数[m2/s]  
定義：$\dot m_m = -\rho_0 D_w・grad\space u$  

$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に対する液相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu l}・grad\space \mu$

#### 空気の拡散係数  
$k_a$：透気率[m2/Ks]  


※新しい基準で物性値を測定した場合、本クラスに書き加えること


#### 水の気体定数

In [5]:
def Rv():
    return 8316.0/18.016

#### 水の密度

In [6]:
def roww():
    return 1000.0

#### 重力加速度

In [ ]:
def grav():
    return 9.806650

$C_r$：水の比熱(specific heat of water)[J/(kg・K)]

In [7]:
def Cr():
    return 4.18605E+3

#### 水蒸気圧を水分化学ポテンシャルで偏微分したもの  $\frac{\partial P_v}{\partial \mu}$

In [8]:
def getDPDM( temp, miu ):
    return vp.Pvs( temp ) * np.exp( miu / Rv() / temp ) / Rv() / temp

#### 水蒸気圧を温度で偏微分したもの  $\frac{\partial P_v}{\partial T}$

In [9]:
def getDPDT( temp, miu ):
    return np.exp( miu / Rv() / temp ) * ( vp.DPvs( temp ) - vp.Pvs( temp ) * miu / Rv() / ( temp**2 ) )

#### 材料の実質部の熱容量に液水の熱容量を加える関数

In [10]:
def getCRow_addWater( crow, phi ):
    return crow + Cr() * roww() * phi

In [15]:
class Kernel_TransferCoefficient():
    # このクラスは Material そのものを表すのではなくて、あくまで CROW 等の物性を計算するアルゴリズムを提供するだけなので、
    # 名前に Kernel とつけました。
    
    def __init__( self, name ):
        
        # 物性名称からインポートすべきモジュール名称を検索する。
        moduleName = getModuleName( name )
        
        # インポートすべきモジュールをインポートする。モジュールが存在しなければエラー。
        module = importModule( moduleName )
        
        # 熱伝導率（固体実質部）
        if( 'getLAM' in dir( module ) ):
            self.getLAM = module.getLAM
        else:
            pass

        # 液相水分伝導率
        if( 'getDw' in dir( module ) ):
            self.getDw = module.getDw
        else:
            pass

        if( 'getLDML' in dir(module) ):
            self.getLDML = module.getLDML
        elif( 'getDw' in dir( module ) ):
            # 本クラスに Dw を計算する式が存在すれば、LDML の計算を Dw を用いて行う。
            # getDwByRH が定義されていれば、getDwByMiu も定義されていれはずなので、ここでは getDwByMiu 関数のみ検索する。
            self.getLDML = lambda c : roww() / grav() * self.getDw( c )
        else:
            pass        
        
        # 気相水分伝導率
        if( 'getDP' in dir( module ) ):
            self.getDP = module.getDP
        elif( 'getLDMG' in dir( module ) and 'getLDTG' in dir(module) ):
            self.getDP = lambda c : ( self.getLDMG( c ) / getDPDM( c.temp, c.miu )  +  self.getLDTG( c ) / getDPDT( c.temp, c.miu ) ) / 2.0
        else:
            pass
        
        if( 'getLDMG' in dir( module ) ):
            self.getLDMG = module.getLDMG
        elif( 'getDP' in dir( module ) ):
            # 本クラスに DP を計算する式が存在すれば、LDMG の計算を DP を用いて行う。
            # getDPByRH が定義されていれば、getDPByMiu も定義されていれはずなので、ここでは getDPByMiu 関数のみ検索する。
            self.getLDMG = lambda c : getDPDM( c.temp, c.miu ) * self.getDP( c )
        else:
            pass
        
        if( 'getLDTG' in dir(module) ):
            self.getLDTG = module.getLDTG
        elif( 'getDP' in dir( module ) ):
            # 本クラスに DP を計算する式が存在すれば、LDTG の計算を DP を用いて行う。
            # getDPByRH が定義されていれば、getDPByMiu も定義されていれはずなので、ここでは getDPByMiu 関数のみ検索する。
            self.getLDTG = lambda c : getDPDT( c.temp, c.miu ) * self.getDP( c )
        else:
            pass


## 使用例

In [16]:
# 使用例
material_prop = Kernel_MaterialProperty('Concrete')
water_prop = Kernel_Material_WaterAdsorption('Concrete')
trans_coeff = Kernel_TransferCoefficient('Concrete')

In [17]:
import liquid_water
import vapour_pressure as vp
c = liquid_water.MiuBasedWater( temp = 293.15, miu = -10.0 )
c.set_material(material_name = 'Concrete')

In [18]:
material_prop.getCRow()

2533520.0

In [19]:
water_prop.getPhi( c )

0.155014590963049

In [20]:
trans_coeff.getLDML( c )

8.845623961863231e-10

In [21]:
trans_coeff .getLDTG( c )

3.9191568164576734e-14

In [22]:
trans_coeff .getLDMG( c )

4.675339581854365e-18

In [23]:
trans_coeff .getDP( c )

2.7053443725309775e-16